In [1]:
import re #pacchetto per espressioni regolari
import os #pacchetto per muoversi nelle cartelle
import sys
sys.path.append('..')  # Add parent directory to the Python path
from utils.helpers import *


In [6]:
train_path = 'data/training.txt'
test_path = 'data/test.txt'

out_dir = 'data/profiling_input'

In [3]:
split = get_split(train_path)
split

'training'

In [4]:

# Necessario eitchettare il blind test

# Read gold.txt file and extract gender values
with open("data/gold.txt", "r") as gold_file:
    gold_data = gold_file.readlines()

# Extract gender labels
gender_list = [line.split()[1].strip() for line in gold_data]

# Read test.txt file
with open("data/test.txt", "r") as test_file:
    test_data = test_file.readlines()

# Replace "?" with gender labels sequentially
updated_test_data = []
gender_index = 0  # Index to track which gender label to use
in_doc = False
for line in test_data:
    if '<doc id="' in line:
        in_doc = True
        if '?' in line:
            label = gender_list[gender_index]
            updated_line = line.replace("?", label)
            gender_index += 1
        else:
            updated_line = line
    elif "</doc>" in line:
        in_doc = False
        updated_line = line
    elif in_doc:
        # If inside a <doc>...</doc> block, continue appending the lines
        updated_line = line.replace("?", label)
    else:
        updated_line = line
    updated_test_data.append(updated_line)

# Write the updated test data to a new file
with open("data/updated_test.txt", "w") as updated_test_file:
    updated_test_file.writelines(updated_test_data)

In [7]:
for file_path in [train_path, test_path]:
    split = get_split(file_path)

    for line in open(file_path, 'r'):
        line = line.strip() # rimuoviamo eventuali spazi e ritorni a capo

        if '<doc id="' in line:    # troviamo un nuovo documento
            doc_id, genre, gender = get_doc_info(line)
            out_file_name = f'{split}#{doc_id}#{genre}#{gender}.txt'
            out_file_path = os.path.join(out_dir, out_file_name)
            user_post = []

        elif '</doc>' in line:     # fine dell'utente
            with open(out_file_path, 'w', encoding="utf-8") as out_file:
                for post in user_post:
                    out_file.write(post)
                    out_file.write('\n')

        elif line != '':  # siamo in corrispondenza del testo di un post
            user_post.append(line)

In [10]:
import os

def detect_strange_characters_in_folder(folder_path):
    strange_filenames = {}
    
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            for char in text:
                if ord(char) > 127:  # Check if character is outside ASCII range
                    if file_name not in strange_filenames:
                        strange_filenames[file_name] = []
                    strange_filenames[file_name].append(char)
    
    return strange_filenames

# Folder containing text files
folder_path = 'data/profiling_input'

# Detect filenames containing strange characters in the folder
strange_filenames = detect_strange_characters_in_folder(folder_path)

# Print the filenames containing strange characters and the strange characters found
print("Files containing strange characters:")
for file_name, strange_chars in strange_filenames.items():
    print("Filename:", file_name)
    print("Strange characters:", strange_chars)


Files containing strange characters:
Filename: test#1007#youtube#F.txt
Strange characters: ['è']
Filename: test#1018#youtube#F.txt
Strange characters: ['ì', 'è']
Filename: test#102#youtube#F.txt
Strange characters: ['à', 'è', 'é']
Filename: test#1028#youtube#M.txt
Strange characters: ['è']
Filename: test#1030#youtube#M.txt
Strange characters: ['ì']
Filename: test#1033#youtube#F.txt
Strange characters: ['à']
Filename: test#1051#youtube#F.txt
Strange characters: ['è']
Filename: test#1052#youtube#F.txt
Strange characters: ['♥']
Filename: test#1054#youtube#F.txt
Strange characters: ['í']
Filename: test#1079#youtube#M.txt
Strange characters: ['“', '”']
Filename: test#11#youtube#F.txt
Strange characters: ['é', '’', 'é', 'ù']
Filename: test#1102#youtube#M.txt
Strange characters: ['ù']
Filename: test#1110#youtube#F.txt
Strange characters: ['é']
Filename: test#1111#youtube#M.txt
Strange characters: ['à']
Filename: test#1114#youtube#M.txt
Strange characters: ['ò', '’']
Filename: test#1122#youtub